In [117]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore
	MarketBasket.ipynb
	credentials/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
#!pip install googletrans==4.0.0rc1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json

from googletrans import Translator

In [ ]:
df = pd.read_csv("datasets/Groceries_dataset.csv")

with open("config/github_access.json") as f:
  acces_to_github = json.load(f)

In [ ]:
df.shape

(38765, 3)

In [ ]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [ ]:
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
df["itemDescription"].unique().tolist()

In [ ]:
translator = Translator()
dic_f = dict()
lista_items = df["itemDescription"].unique().tolist()
for item in lista_items:
  try:
    trans = translator.translate(item, src="en", dest='es').text
  except:
    trans = item  
    print(f"error en {item}")
  dic_f[item] = trans

error en pork
error en turkey
error en cleaner


In [ ]:
dic_f["pork"] = "cerdo"
dic_f["turkey"] = "pavo"
dic_f["cleaner"] = "limpiador"

In [ ]:
df_clean = df.replace({"itemDescription":dic_f})

In [ ]:
class Processingdata:

  def __init__(self,df):
    self.df_original = df
  
  def transform_columns(self,df):
    
    # str to column
    df["Date"] = pd.to_datetime(df["Date"]) 

    #translate items
    



,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-05-01,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-01-02,whole milk
...,...,...,...
38760,4471,2014-08-10,sliced cheese
38761,2022,2014-02-23,candy
38762,1097,2014-04-16,cake bar
38763,1510,2014-03-12,fruit/vegetable juice


In [ ]:
df_clean.to_csv("datasets/groceries_clean.csv")

In [ ]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore
	MarketBasket.ipynb
	credentials/
	datasets/

nothing added to commit but untracked files present (use "git add" to track)
